# Prepare the notebook for use

## Install packages & import libraries

In [1]:
# Download required packages to Colab
!pip install -q langchain
!pip install -q openai
!pip install -q chromadb
!pip install -q tiktoken
!pip install -q duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.8/496.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.2 MB/s eta 

In [2]:
# Import required libraries
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, RetrievalQA
from langchain import ConversationChain
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.tools import DuckDuckGoSearchRun
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import os
from getpass import getpass

from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, AIMessage, SystemMessage

import pandas as pd
import csv

## OpenAI API key

In [3]:
openai_api_key = getpass()
os.environ["OPENAI_API_KEY"] = openai_api_key

··········


## Connect Google Drive

## Read data

1. [var_comb] varibale_combination.csv
2. [prompt_result] answer_evaluation_prompt_result.csv
3. [qa_comb] interview_question_answer_combination.csv
4. [how_to_answer] how_to_answer_behavioral_question.txt
5. [company_looking] what_are_things_companies_looking_for_during_behavioral_interview.txt

Since the results we got from LLMs will be different everytime, we want to use .csv files to track those results which could help use go back to see what we can do or what we did to improve the outputs.

In [23]:
var_comb = pd.read_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/variable_combination.csv")
prompt_result = pd.read_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/answer_evaluation_prompt_result.csv")
qa_comb = pd.read_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/interview_question_answer_combination.csv")

In [24]:
var_comb

,comb_num,interviewer_role,company_name,resume,role_title,role_description,company_description
0,1,recruiter,S&P Global,\nEDUCATION\nVanderbilt University – Data Scie...,Data Engineer - AI/ML (Virtual),\nJob Description\n\nThis is a software develo...,\nOverview\nS&P Global (NYSE: SPGI) provides e...


In [25]:
prompt_result

,model,question_answer_combination,variable_combination,template,output
0,gpt_4,1,1,\nYou are an expert evaluator of interview res...,Relevance to Question: \nScore: 10/10 \nFeedba...
1,gpt_4,2,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe candidate's ...
2,gpt_4,3,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 9/10\nThe candidate dir...
3,gpt_4,5,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 7/10\nThe candidate doe...
4,gpt_4,12,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 8/10\nThe candidate doe...
5,gpt_4,23,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe answer direc...


In [26]:
qa_comb

,comb_num,var_comb_num,interview_question,question_type,question_answer
0,1,1,Describe a challenging project you worked on. ...,behavioral,"\nIn my last role as a project manager, I led ..."
1,2,1,Describe a challenging project you worked on. ...,behavioral,\nI was part of a marketing campaign that face...
2,3,1,Describe a challenging project you worked on. ...,behavioral,"\nAs a junior analyst, I worked on a data migr..."
3,4,1,Describe a challenging project you worked on. ...,behavioral,"\nI worked on a challenging project last year,..."
4,5,1,Describe a challenging project you worked on. ...,behavioral,"\nIn my previous job, there was a project that..."
5,6,1,Describe a challenging project you worked on. ...,behavioral,\nI was involved in a project that didn't go w...
6,7,1,Describe a challenging project you worked on. ...,behavioral,\nI led a cross-functional team on a critical ...
7,8,1,Describe a challenging project you worked on. ...,behavioral,"\nIn my role as an IT specialist, I was tasked..."
8,9,1,How do you handle disagreements with colleagues?,behavioral,\nI believe in open communication and empathy....
9,10,1,How do you handle disagreements with colleagues?,behavioral,\nDisagreements can be challenging. I usually ...


## Define Parameters

In [27]:
var_comb_num = 1 # this means we use the 1st combination of information in var_comb.csv dataset; this helps us tack information

In [47]:
file1 = open("/content/drive/MyDrive/Capstone Project - Mock Interview App/data/how_to_answer_behavioral_question.txt", "r")
file2 = open("/content/drive/MyDrive/Capstone Project - Mock Interview App/data/what_are_things_companies_looking_for_during_behavioral_interview.txt", "r")

In [48]:
how_to_answer = file1.read()
company_looking = file2.read()

In [35]:
print(how_to_answer)
print("===============")
print(company_looking)

When answering behavioral interview questions, it's essential to present your qualifications effectively by incorporating specific aspects that highlight your skills, experiences, and personal attributes. These aspects are crucial in demonstrating your suitability for the role and how well you align with the organization's values and culture. Here are key aspects to include:
1.	Specific Examples: Use concrete examples from your past experiences to illustrate how you've handled situations relevant to the question. This provides tangible evidence of your skills and abilities.
2.	STAR Method: Structure your response using the STAR method – Situation, Task, Action, and Result. This helps in organizing your answer clearly and concisely:
•	Situation: Describe the context or background of the example you're using.
•	Task: Explain the task or challenge you were facing.
•	Action: Detail the specific actions you took to address the task or challenge.
•	Result: Share the outcomes of your actions,

# Model selection

In [42]:
chat = ChatOpenAI(model_name = "gpt-4")

In [43]:
# record model as ex_model
ex_model = "gpt_4"

# Answer Evaluation Prompt

## Prompt 2.1 (var without detail)

In [29]:
# set up input variables for prompt
input_var_2 = ["interview_questions",
               "role_title",
               "company_name",
               "question_answers"] #"role_description", "company_description"]

In [59]:
template_2 = """
As an experienced career coach, you help students improve their skills for
answering behavioral interview questions.  Please help this student improve the
answer to the behavioral interview question {interview_questions} during the
interview of {role_title} at {company_name}.

The student’s answer: {question_answers}

First, give the student's answer a score, and point by point feedback with rationale.
Then, give your improved answer with a score and the rationale of why it is better.

Here are aspects that how interviewers should answer a behavioral interview question:
When answering behavioral interview questions, it's essential to present your qualifications effectively by incorporating specific aspects that highlight your skills, experiences, and personal attributes. These aspects are crucial in demonstrating your suitability for the role and how well you align with the organization's values and culture. Here are key aspects to include:
1.	Specific Examples: Use concrete examples from your past experiences to illustrate how you've handled situations relevant to the question. This provides tangible evidence of your skills and abilities.
2.	STAR Method: Structure your response using the STAR method – Situation, Task, Action, and Result. This helps in organizing your answer clearly and concisely:
•	Situation: Describe the context or background of the example you're using.
•	Task: Explain the task or challenge you were facing.
•	Action: Detail the specific actions you took to address the task or challenge.
•	Result: Share the outcomes of your actions, highlighting your contributions and any lessons learned.
3.	Relevance to the Role: Tailor your examples to demonstrate skills and attributes that are directly relevant to the job you're applying for. Show how your past experiences have prepared you for this specific role.
4.	Problem-Solving Skills: Highlight your ability to identify problems, think critically, and implement effective solutions. Employers value candidates who can tackle challenges proactively.
5.	Teamwork and Collaboration: Discuss how you work with others, emphasizing teamwork, communication, and interpersonal skills, especially if the role involves working closely with a team.
6.	Adaptability and Flexibility: Show your ability to adapt to new situations, learn from experiences, and remain flexible in the face of change or uncertainty.
7.	Leadership and Initiative: For roles that require leadership, illustrate your ability to lead, inspire, and motivate others. Even for non-leadership roles, showing initiative and the ability to take charge in appropriate situations can be beneficial.
8.	Results and Impact: Focus on the results and impact of your actions. Quantify your achievements where possible, as this adds credibility and a sense of scale to your accomplishments.
9.	Personal Development: Reflect on what you learned from the experience and how it contributed to your personal and professional growth.
10.	Cultural Fit: Convey how your values, work ethic, and personality align with the company's culture. Show that you not only have the skills for the job but also would thrive in the organization's environment.
Including these aspects in your behavioral interview responses can significantly strengthen your answers, showcasing your qualifications, experiences, and fit for the role.

Here are aspects that companies hope to get from your answer of a behavioral question:
Companies conduct behavioral interviews to assess various aspects of a candidate's past behavior, skills, and performance to predict their future behavior and suitability for the role in their organization. Here are some key elements that companies typically look for during behavioral interviews:
1.	Past Performance as an Indicator of Future Behavior: Companies believe that past behavior is a good predictor of future behavior. By understanding how you handled situations in the past, they can gauge how you might perform in similar situations in their organization.
2.	Problem-Solving and Decision-Making Skills: Employers look for candidates who can demonstrate effective problem-solving and decision-making skills. They are interested in how you approach challenges, analyze problems, and arrive at solutions.
3.	Teamwork and Collaboration: Since most roles require some level of teamwork, companies want to see how well you work with others. Your ability to collaborate, communicate effectively, and contribute to a team is crucial.
4.	Adaptability and Flexibility: With the fast pace of change in the workplace, companies seek candidates who can adapt to new situations and challenges quickly and effectively.
5.	Leadership Qualities: For roles that involve leading others, employers look for leadership potential. This includes your ability to motivate, guide, and influence others, even if the role is not a traditional leadership position.
6.	Work Ethic and Professionalism: Your approach to work, commitment to quality, and professional conduct are important to employers. They want to ensure you have a strong work ethic and can represent the company positively.
7.	Cultural Fit: Companies are keen on finding candidates who will fit well with their organizational culture. This includes sharing similar values, work styles, and the ability to thrive in the company’s work environment.
8.	Resilience and Stress Management: Employers may look for signs of resilience and your ability to handle stress or failure. They want employees who can remain productive and positive, even in challenging situations.
9.	Communication Skills: Effective communication is key in any role. Companies assess your ability to articulate ideas clearly, listen to others, and communicate in a way that is effective and appropriate for the workplace.
10.	Initiative and Self-motivation: Demonstrating that you are a self-starter who takes initiative is highly valued. Companies prefer candidates who show they can work independently and are motivated to achieve.
In summary, behavioral interviews are designed to evaluate a candidate's suitability not only in terms of skills and experience but also in terms of their personality, work habits, and potential fit within the company's culture and team dynamics.
"""

In [60]:
prompt_21 = PromptTemplate(
    input_variables = input_var_2,
    template = template_2
)

### Code

In [51]:
job_title = var_comb.role_title[0]
comp_name = var_comb.company_name[0]

job_des = var_comb.role_description[0]
comp_des = var_comb.company_description[0]

#### Q1 A1 [8/10]

In [52]:
# set up prameters
qa_comb_num = 1
inter_q = qa_comb.interview_question[0]
q_a = qa_comb.question_answer[0]

In [61]:
#from pydantic.typing import resolve_annotations

new_prompt_21 = prompt_21.format(interview_questions = inter_q,
                                 role_title = job_title,
                                 company_name = comp_name,
                                 question_answers = q_a) #role_description = job_des, company_description = comp_des)
result = chat([HumanMessage(content=new_prompt_21)])

In [62]:
print(result.content)

Score: 8/10

Feedback: 
1. Specific Examples: The student did well in providing a specific example of a challenging project that he led. It's clear and relevant to the role he's applying for. 
2. STAR Method: The student used the STAR method but could have provided more detail on the situation, task and actions to present a more comprehensive picture of the project.
3. Relevance to the Role: The example was relevant to the role and demonstrated project management and problem-solving skills.
4. Problem-Solving Skills: The student showed problem-solving skills through brainstorming sessions and delegating tasks.
5. Teamwork and Collaboration: The student highlighted his teamwork and collaboration skills but could have provided more details on how he worked with his team.
6. Adaptability and Flexibility: The student showed adaptability by handling unexpected technical hurdles but could have elaborated more on this.
7. Leadership and Initiative: The student demonstrated leadership by leadi

##### store output

In [64]:
# ex_model, qa_comb_num, var_comb_num, template_2, result.content

new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'question_answer_combination': [qa_comb_num],
    'variable_combination': [var_comb_num],
    'template': [template_2],
    'output': [result.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

In [65]:
prompt_result

,model,question_answer_combination,variable_combination,template,output
0,gpt_4,1,1,\nYou are an expert evaluator of interview res...,Relevance to Question: \nScore: 10/10 \nFeedba...
1,gpt_4,2,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe candidate's ...
2,gpt_4,3,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 9/10\nThe candidate dir...
3,gpt_4,5,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 7/10\nThe candidate doe...
4,gpt_4,12,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 8/10\nThe candidate doe...
5,gpt_4,23,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe answer direc...
6,gpt_4,1,1,"\nAs an experienced career coach, you help stu...",Score: 8/10\n\nFeedback: \n1. Specific Example...


#### Q1 A5 [2/10]

In [71]:
# set up prameters
question_num = 1
answer_num = 5
ind = (question_num-1) * 8 + answer_num - 1


qa_comb_num = qa_comb.comb_num[ind]

inter_q = qa_comb.interview_question[ind]
q_a = qa_comb.question_answer[ind]

In [72]:
inter_q

'Describe a challenging project you worked on. What was your role and how did you handle it?'

In [73]:
q_a

"\nIn my previous job, there was a project that was really tough. I found it \noverwhelming, so I mostly kept to the tasks I was comfortable with and didn't \nventure much beyond that.\n"

In [74]:
new_prompt = prompt_21.format(interview_questions = inter_q,
                                 role_title = job_title,
                                 company_name = comp_name,
                                 question_answers = q_a) #role_description = job_des, company_description = comp_des)
result = chat([HumanMessage(content=new_prompt)])

In [75]:
print(result.content)

Student's Answer Score: 2/10

Feedback:
1. Specific Examples: The student does mention a challenging project, but does not provide specific examples that highlight their skills or experiences.
2. STAR Method: The student's response does not follow the STAR method. There is a mention of a situation, but the task, action, and result are not clearly defined.
3. Relevance to the Role: The response does not show how the experience is relevant to the role of Data Engineer - AI/ML.
4. Problem-Solving Skills: The student does not demonstrate problem-solving skills as they admit to only sticking to tasks they were comfortable with.
5. Teamwork and Collaboration: There is no mention of teamwork or collaboration.
6. Adaptability and Flexibility: There is a lack of adaptability and flexibility as the student did not venture beyond their comfort zone.
7. Leadership and Initiative: The response does not show any leadership skills or initiative.
8. Results and Impact: The student does not mention any

##### store output

In [76]:
# ex_model, qa_comb_num, var_comb_num, template_2, result.content

new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'question_answer_combination': [qa_comb_num],
    'variable_combination': [var_comb_num],
    'template': [template_2],
    'output': [result.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

In [77]:
prompt_result

,model,question_answer_combination,variable_combination,template,output
0,gpt_4,1,1,\nYou are an expert evaluator of interview res...,Relevance to Question: \nScore: 10/10 \nFeedba...
1,gpt_4,2,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe candidate's ...
2,gpt_4,3,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 9/10\nThe candidate dir...
3,gpt_4,5,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 7/10\nThe candidate doe...
4,gpt_4,12,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 8/10\nThe candidate doe...
5,gpt_4,23,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe answer direc...
6,gpt_4,1,1,"\nAs an experienced career coach, you help stu...",Score: 8/10\n\nFeedback: \n1. Specific Example...
7,gpt_4,5,1,"\nAs an experienced career coach, you help stu...",Student's Answer Score: 2/10\n\nFeedback:\n1. ...


#### Q3 A7 [7/10]

In [78]:
# set up prameters
question_num = 3
answer_num = 7
ind = (question_num-1) * 8 + answer_num - 1

qa_comb_num = qa_comb.comb_num[ind]

inter_q = qa_comb.interview_question[ind]
q_a = qa_comb.question_answer[ind]

In [79]:
inter_q

'How do you stay motivated and manage your time effectively?'

In [80]:
q_a

'\nMy motivation comes from setting challenging yet achievable goals. I use a \ncombination of the Eisenhower Matrix for prioritizing tasks and the Pomodoro \nTechnique for focused work sessions. This approach helped me successfully manage \nmultiple projects simultaneously, leading to a 20% increase in efficiency in my \ndepartment.\n'

In [81]:
new_prompt = prompt_21.format(interview_questions = inter_q,
                                 role_title = job_title,
                                 company_name = comp_name,
                                 question_answers = q_a) #role_description = job_des, company_description = comp_des)
result = chat([HumanMessage(content=new_prompt)])

In [82]:
print(result.content)

Student's answer score: 7/10

Feedback:
1. Specific Examples: The student provided a concrete example of using the Eisenhower Matrix and the Pomodoro Technique to manage time and stay motivated. This gives a clear picture of the methods they use. Excellent job on this aspect.
2. STAR Method: The student used this method to an extent, but could have been more specific about the situation and task that required the use of these techniques, and the actions taken to implement them.
3. Relevance to the Role: The student's answer is relevant for a Data Engineer role where time management and efficiency are key. However, they could have highlighted more about how these skills are especially relevant to data engineering work.
4. Problem-Solving Skills: The student did not explicitly mention problem-solving skills. These skills are crucial for a Data Engineer role, and should be added.
5. Teamwork and Collaboration: The student did not mention teamwork or collaboration, which are also important

##### store output

In [83]:
# ex_model, qa_comb_num, var_comb_num, template_2, result.content

new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'question_answer_combination': [qa_comb_num],
    'variable_combination': [var_comb_num],
    'template': [template_2],
    'output': [result.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

In [84]:
prompt_result

,model,question_answer_combination,variable_combination,template,output
0,gpt_4,1,1,\nYou are an expert evaluator of interview res...,Relevance to Question: \nScore: 10/10 \nFeedba...
1,gpt_4,2,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe candidate's ...
2,gpt_4,3,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 9/10\nThe candidate dir...
3,gpt_4,5,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 7/10\nThe candidate doe...
4,gpt_4,12,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 8/10\nThe candidate doe...
5,gpt_4,23,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe answer direc...
6,gpt_4,1,1,"\nAs an experienced career coach, you help stu...",Score: 8/10\n\nFeedback: \n1. Specific Example...
7,gpt_4,5,1,"\nAs an experienced career coach, you help stu...",Student's Answer Score: 2/10\n\nFeedback:\n1. ...
8,gpt_4,23,1,"\nAs an experienced career coach, you help stu...",Student's answer score: 7/10\n\nFeedback:\n1. ...


## Prompt 2.2 (var with detail)

In [85]:
# set up input variables for prompt
input_var_3 = ["interview_questions",
               "role_title",
               "company_name",
               "question_answers",
               "role_description", "company_description"]

In [86]:
template_3 = """
As an experienced career coach, you help students improve their skills for
answering behavioral interview questions.  Please help this student improve the
answer to the behavioral interview question {interview_questions} during the
interview of {role_title} at {company_name}.

The student’s answer: {question_answers}

First, give the student's answer a score, and point by point feedback with rationale.
Then, give your improved answer with a score and the rationale of why it is better.

Here are job description and company backgroound that you might want to consider:
Job Description: {role_description}
Company Background: {company_description}

Here are aspects that how interviewers should answer a behavioral interview question:
When answering behavioral interview questions, it's essential to present your qualifications effectively by incorporating specific aspects that highlight your skills, experiences, and personal attributes. These aspects are crucial in demonstrating your suitability for the role and how well you align with the organization's values and culture. Here are key aspects to include:
1.	Specific Examples: Use concrete examples from your past experiences to illustrate how you've handled situations relevant to the question. This provides tangible evidence of your skills and abilities.
2.	STAR Method: Structure your response using the STAR method – Situation, Task, Action, and Result. This helps in organizing your answer clearly and concisely:
•	Situation: Describe the context or background of the example you're using.
•	Task: Explain the task or challenge you were facing.
•	Action: Detail the specific actions you took to address the task or challenge.
•	Result: Share the outcomes of your actions, highlighting your contributions and any lessons learned.
3.	Relevance to the Role: Tailor your examples to demonstrate skills and attributes that are directly relevant to the job you're applying for. Show how your past experiences have prepared you for this specific role.
4.	Problem-Solving Skills: Highlight your ability to identify problems, think critically, and implement effective solutions. Employers value candidates who can tackle challenges proactively.
5.	Teamwork and Collaboration: Discuss how you work with others, emphasizing teamwork, communication, and interpersonal skills, especially if the role involves working closely with a team.
6.	Adaptability and Flexibility: Show your ability to adapt to new situations, learn from experiences, and remain flexible in the face of change or uncertainty.
7.	Leadership and Initiative: For roles that require leadership, illustrate your ability to lead, inspire, and motivate others. Even for non-leadership roles, showing initiative and the ability to take charge in appropriate situations can be beneficial.
8.	Results and Impact: Focus on the results and impact of your actions. Quantify your achievements where possible, as this adds credibility and a sense of scale to your accomplishments.
9.	Personal Development: Reflect on what you learned from the experience and how it contributed to your personal and professional growth.
10.	Cultural Fit: Convey how your values, work ethic, and personality align with the company's culture. Show that you not only have the skills for the job but also would thrive in the organization's environment.
Including these aspects in your behavioral interview responses can significantly strengthen your answers, showcasing your qualifications, experiences, and fit for the role.

Here are aspects that companies hope to get from your answer of a behavioral question:
Companies conduct behavioral interviews to assess various aspects of a candidate's past behavior, skills, and performance to predict their future behavior and suitability for the role in their organization. Here are some key elements that companies typically look for during behavioral interviews:
1.	Past Performance as an Indicator of Future Behavior: Companies believe that past behavior is a good predictor of future behavior. By understanding how you handled situations in the past, they can gauge how you might perform in similar situations in their organization.
2.	Problem-Solving and Decision-Making Skills: Employers look for candidates who can demonstrate effective problem-solving and decision-making skills. They are interested in how you approach challenges, analyze problems, and arrive at solutions.
3.	Teamwork and Collaboration: Since most roles require some level of teamwork, companies want to see how well you work with others. Your ability to collaborate, communicate effectively, and contribute to a team is crucial.
4.	Adaptability and Flexibility: With the fast pace of change in the workplace, companies seek candidates who can adapt to new situations and challenges quickly and effectively.
5.	Leadership Qualities: For roles that involve leading others, employers look for leadership potential. This includes your ability to motivate, guide, and influence others, even if the role is not a traditional leadership position.
6.	Work Ethic and Professionalism: Your approach to work, commitment to quality, and professional conduct are important to employers. They want to ensure you have a strong work ethic and can represent the company positively.
7.	Cultural Fit: Companies are keen on finding candidates who will fit well with their organizational culture. This includes sharing similar values, work styles, and the ability to thrive in the company’s work environment.
8.	Resilience and Stress Management: Employers may look for signs of resilience and your ability to handle stress or failure. They want employees who can remain productive and positive, even in challenging situations.
9.	Communication Skills: Effective communication is key in any role. Companies assess your ability to articulate ideas clearly, listen to others, and communicate in a way that is effective and appropriate for the workplace.
10.	Initiative and Self-motivation: Demonstrating that you are a self-starter who takes initiative is highly valued. Companies prefer candidates who show they can work independently and are motivated to achieve.
In summary, behavioral interviews are designed to evaluate a candidate's suitability not only in terms of skills and experience but also in terms of their personality, work habits, and potential fit within the company's culture and team dynamics.
"""

In [87]:
prompt_22 = PromptTemplate(
    input_variables = input_var_3,
    template = template_3
)

### Code

In [88]:
job_title = var_comb.role_title[0]
comp_name = var_comb.company_name[0]

job_des = var_comb.role_description[0]
comp_des = var_comb.company_description[0]

#### Q1 A1 [7/10]

In [89]:
# set up prameters
qa_comb_num = 1
inter_q = qa_comb.interview_question[0]
q_a = qa_comb.question_answer[0]

In [90]:
#from pydantic.typing import resolve_annotations

new_prompt = prompt_22.format(interview_questions = inter_q,
                                 role_title = job_title,
                                 company_name = comp_name,
                                 question_answers = q_a,
                              role_description = job_des, company_description = comp_des)
result = chat([HumanMessage(content=new_prompt)])

In [91]:
print(result.content)

Student's Answer Score: 7/10

Feedback with rationale:

1. Specific Examples: The student provided a concrete example of a challenging project they handled in the past. This is good as it gives the interviewer a clear picture of their experience and problem-solving skills. 

2. STAR Method: The student used the STAR method to structure their response. They described the situation (high-stakes software development project with a tight deadline and unexpected technical hurdles), the task (leading the team), the action (organizing extra brainstorming sessions, delegating tasks, and maintaining open communication), and the result (delivering the project on time and receiving positive feedback from the client). This structure makes the answer clear and easy to follow.

3. Relevance to the Role: The student could have done a better job of tailoring their answer to the specific role of a Data Engineer - AI/ML. They did not mention any relevant skills like implementing machine learning algorit

##### store output

In [92]:
# ex_model, qa_comb_num, var_comb_num, template_3, result.content

new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'question_answer_combination': [qa_comb_num],
    'variable_combination': [var_comb_num],
    'template': [template_3],
    'output': [result.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

In [93]:
prompt_result

,model,question_answer_combination,variable_combination,template,output
0,gpt_4,1,1,\nYou are an expert evaluator of interview res...,Relevance to Question: \nScore: 10/10 \nFeedba...
1,gpt_4,2,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe candidate's ...
2,gpt_4,3,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 9/10\nThe candidate dir...
3,gpt_4,5,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 7/10\nThe candidate doe...
4,gpt_4,12,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 8/10\nThe candidate doe...
5,gpt_4,23,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe answer direc...
6,gpt_4,1,1,"\nAs an experienced career coach, you help stu...",Score: 8/10\n\nFeedback: \n1. Specific Example...
7,gpt_4,5,1,"\nAs an experienced career coach, you help stu...",Student's Answer Score: 2/10\n\nFeedback:\n1. ...
8,gpt_4,23,1,"\nAs an experienced career coach, you help stu...",Student's answer score: 7/10\n\nFeedback:\n1. ...
9,gpt_4,1,1,"\nAs an experienced career coach, you help stu...",Student's Answer Score: 7/10\n\nFeedback with ...


#### Q1 A5 [3/10]

In [94]:
# set up prameters
question_num = 1
answer_num = 5
ind = (question_num-1) * 8 + answer_num - 1


qa_comb_num = qa_comb.comb_num[ind]

inter_q = qa_comb.interview_question[ind]
q_a = qa_comb.question_answer[ind]

In [95]:
inter_q

'Describe a challenging project you worked on. What was your role and how did you handle it?'

In [96]:
q_a

"\nIn my previous job, there was a project that was really tough. I found it \noverwhelming, so I mostly kept to the tasks I was comfortable with and didn't \nventure much beyond that.\n"

In [97]:
new_prompt = prompt_22.format(interview_questions = inter_q,
                                 role_title = job_title,
                                 company_name = comp_name,
                                 question_answers = q_a,
                              role_description = job_des, company_description = comp_des)
result = chat([HumanMessage(content=new_prompt)])

In [98]:
print(result.content)

Student's Answer Score: 3/10

Feedback:
The student's response lacks detail and doesn't provide a good understanding of their role in the project, the specific challenges they faced, or how they handled them. It also doesn't show any demonstration of skills or experiences relevant to the Data Engineer - AI/ML role at S&P Global. The response also fails to mention any outcomes or achievements, making it difficult to assess their competency or performance. Furthermore, the student's answer indicates a lack of initiative and adaptability, which are important qualities for the role in question.

Improved Answer:

In a previous role as a data engineer, I was tasked with the challenge of developing an AI model for predicting customer churn for a telecom company. I was the lead data engineer responsible for the entire lifecycle of the project, from data collection and cleaning to implementing machine learning algorithms.

The project was challenging due to the large volume of data and the com

##### store output

In [99]:
# ex_model, qa_comb_num, var_comb_num, template_3, result.content

new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'question_answer_combination': [qa_comb_num],
    'variable_combination': [var_comb_num],
    'template': [template_3],
    'output': [result.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

In [100]:
prompt_result

,model,question_answer_combination,variable_combination,template,output
0,gpt_4,1,1,\nYou are an expert evaluator of interview res...,Relevance to Question: \nScore: 10/10 \nFeedba...
1,gpt_4,2,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe candidate's ...
2,gpt_4,3,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 9/10\nThe candidate dir...
3,gpt_4,5,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 7/10\nThe candidate doe...
4,gpt_4,12,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 8/10\nThe candidate doe...
5,gpt_4,23,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe answer direc...
6,gpt_4,1,1,"\nAs an experienced career coach, you help stu...",Score: 8/10\n\nFeedback: \n1. Specific Example...
7,gpt_4,5,1,"\nAs an experienced career coach, you help stu...",Student's Answer Score: 2/10\n\nFeedback:\n1. ...
8,gpt_4,23,1,"\nAs an experienced career coach, you help stu...",Student's answer score: 7/10\n\nFeedback:\n1. ...
9,gpt_4,1,1,"\nAs an experienced career coach, you help stu...",Student's Answer Score: 7/10\n\nFeedback with ...


#### Q3 A7 [8/10]

In [101]:
# set up prameters
question_num = 3
answer_num = 7
ind = (question_num-1) * 8 + answer_num - 1

qa_comb_num = qa_comb.comb_num[ind]

inter_q = qa_comb.interview_question[ind]
q_a = qa_comb.question_answer[ind]

In [102]:
inter_q

'How do you stay motivated and manage your time effectively?'

In [103]:
q_a

'\nMy motivation comes from setting challenging yet achievable goals. I use a \ncombination of the Eisenhower Matrix for prioritizing tasks and the Pomodoro \nTechnique for focused work sessions. This approach helped me successfully manage \nmultiple projects simultaneously, leading to a 20% increase in efficiency in my \ndepartment.\n'

In [104]:
new_prompt = prompt_22.format(interview_questions = inter_q,
                                 role_title = job_title,
                                 company_name = comp_name,
                                 question_answers = q_a,
                              role_description = job_des, company_description = comp_des)
result = chat([HumanMessage(content=new_prompt)])

In [105]:
print(result.content)

Student's Answer Score: 8/10

Feedback:
The student's response is quite good as they demonstrate their ability to set goals, prioritize tasks, and manage time effectively. They also provide a quantifiable achievement, which is always beneficial in interviews. However, they could provide a more specific example to illustrate their approach in action, and could also tie their answer more directly to the role and company they are applying for. 

Improved Answer:
I draw motivation from setting challenging yet achievable goals. I use the Eisenhower Matrix to prioritize tasks according to their urgency and importance. This helps me focus on tasks that are vital for project success and delegate or schedule less critical tasks appropriately. I also use the Pomodoro Technique for focused work sessions, breaking tasks into 25-minute intervals with short breaks in between. This helps me maintain high productivity levels without burning out. 

For example, while working on a complex AI model at my

##### store output

In [106]:
# ex_model, qa_comb_num, var_comb_num, template_3, result.content

new_prompt_result = pd.DataFrame({
    'model': [ex_model],
    'question_answer_combination': [qa_comb_num],
    'variable_combination': [var_comb_num],
    'template': [template_3],
    'output': [result.content]
})

prompt_result = pd.concat([prompt_result, new_prompt_result], ignore_index=True)

In [107]:
prompt_result

,model,question_answer_combination,variable_combination,template,output
0,gpt_4,1,1,\nYou are an expert evaluator of interview res...,Relevance to Question: \nScore: 10/10 \nFeedba...
1,gpt_4,2,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe candidate's ...
2,gpt_4,3,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 9/10\nThe candidate dir...
3,gpt_4,5,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 7/10\nThe candidate doe...
4,gpt_4,12,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 8/10\nThe candidate doe...
5,gpt_4,23,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe answer direc...
6,gpt_4,1,1,"\nAs an experienced career coach, you help stu...",Score: 8/10\n\nFeedback: \n1. Specific Example...
7,gpt_4,5,1,"\nAs an experienced career coach, you help stu...",Student's Answer Score: 2/10\n\nFeedback:\n1. ...
8,gpt_4,23,1,"\nAs an experienced career coach, you help stu...",Student's answer score: 7/10\n\nFeedback:\n1. ...
9,gpt_4,1,1,"\nAs an experienced career coach, you help stu...",Student's Answer Score: 7/10\n\nFeedback with ...


# Compare & Contrast
More details:
Compare & Contrast Answer Evaluation Prompts & Results
(https://docs.google.com/document/d/1WBKhjGyOa_67jFeTWWVvkvCUikkREorMbxOtcF1rKQQ/edit?usp=sharing).


**In conclusion:**
1. Prompt 2 & 3 worked better than prompt 1. They not only provide a score for student's answer, but also give some feedback and a improved answer;
2. Compare to prompt 3, prompt 2 has more detailed feedback for student's original answer from different aspects;
3. Compare to prompt 2, prompt 3 is more aligned with the job and company background.

<font color="Blue">Next step:  </font>  
-[ ] Try to combine result form prompt 2 & prompt 3, so we could have both detailed feedback and answer better aligned with the job and company the student applied.

## Question 1 & Answer 1

In [126]:
# prompt 1/2/3, question 1, example answer 1

p1_q1_a1 = prompt_result.iloc[0,4]

p2_q1_a1 = prompt_result.iloc[6,4]

p3_q1_a1 = prompt_result.iloc[9,4]

In [127]:
print(p1_q1_a1)

Relevance to Question: 
Score: 10/10 
Feedback: The candidate clearly describes a challenging project that they worked on, outlining their role and the strategies they used to handle the situation.

Technical Skills: 
Score: 7/10 
Feedback: The candidate shows good project management skills, but specific technical skills related to AI/ML software development were not clearly mentioned.

Problem-Solving Abilities: 
Score: 9/10 
Feedback: The candidate demonstrates strong problem-solving abilities by organizing extra brainstorming sessions, delegating tasks, and maintaining open communication with stakeholders to meet the project deadline.

Communication Skills: 
Score: 10/10 
Feedback: The candidate effectively communicates their thoughts, outlining the problem and their strategies in a clear, concise manner.

Cultural Fit: 
Score: 8/10 
Feedback: The candidate shows good project management skills and a strong ability to handle pressure, which aligns with S&P Global's fast-paced and dat

In [128]:
print(p2_q1_a1)

Score: 8/10

Feedback: 
1. Specific Examples: The student did well in providing a specific example of a challenging project that he led. It's clear and relevant to the role he's applying for. 
2. STAR Method: The student used the STAR method but could have provided more detail on the situation, task and actions to present a more comprehensive picture of the project.
3. Relevance to the Role: The example was relevant to the role and demonstrated project management and problem-solving skills.
4. Problem-Solving Skills: The student showed problem-solving skills through brainstorming sessions and delegating tasks.
5. Teamwork and Collaboration: The student highlighted his teamwork and collaboration skills but could have provided more details on how he worked with his team.
6. Adaptability and Flexibility: The student showed adaptability by handling unexpected technical hurdles but could have elaborated more on this.
7. Leadership and Initiative: The student demonstrated leadership by leadi

In [129]:
print(p3_q1_a1)

Student's Answer Score: 7/10

Feedback with rationale:

1. Specific Examples: The student provided a concrete example of a challenging project they handled in the past. This is good as it gives the interviewer a clear picture of their experience and problem-solving skills. 

2. STAR Method: The student used the STAR method to structure their response. They described the situation (high-stakes software development project with a tight deadline and unexpected technical hurdles), the task (leading the team), the action (organizing extra brainstorming sessions, delegating tasks, and maintaining open communication), and the result (delivering the project on time and receiving positive feedback from the client). This structure makes the answer clear and easy to follow.

3. Relevance to the Role: The student could have done a better job of tailoring their answer to the specific role of a Data Engineer - AI/ML. They did not mention any relevant skills like implementing machine learning algorit

## Question 1 & Answer 5

In [130]:
# prompt 1/2/3, question 1, example answer 5

p1_q1_a5 = prompt_result.iloc[3,4]

p2_q1_a5 = prompt_result.iloc[7,4]

p3_q1_a5 = prompt_result.iloc[10,4]

In [131]:
print(p1_q1_a5)

Relevance to Question: 7/10
The candidate does speak about a challenging project, which is the main point of the question, but fails to go into any detail about their role or how they specifically handled it. 

Technical Skills: 2/10 
The candidate does not demonstrate any technical skills in their answer, which is a key requirement for the job role.

Problem-Solving Abilities: 3/10 
The candidate did not demonstrate problem-solving abilities. Instead of finding solutions to overcome the challenge, they kept to tasks they were comfortable with.

Communication Skills: 7/10 
The candidate communicates clearly and directly, but the answer lacks depth and details.

Cultural Fit: 4/10
The answer does not provide any indication of alignment or conflict with S&P Global's culture. More information would be required to make a proper assessment.

Adaptability: 3/10 
The candidate does not show adaptability in their response. They stayed within their comfort zone instead of adapting to the projec

In [132]:
print(p2_q1_a5)

Student's Answer Score: 2/10

Feedback:
1. Specific Examples: The student does mention a challenging project, but does not provide specific examples that highlight their skills or experiences.
2. STAR Method: The student's response does not follow the STAR method. There is a mention of a situation, but the task, action, and result are not clearly defined.
3. Relevance to the Role: The response does not show how the experience is relevant to the role of Data Engineer - AI/ML.
4. Problem-Solving Skills: The student does not demonstrate problem-solving skills as they admit to only sticking to tasks they were comfortable with.
5. Teamwork and Collaboration: There is no mention of teamwork or collaboration.
6. Adaptability and Flexibility: There is a lack of adaptability and flexibility as the student did not venture beyond their comfort zone.
7. Leadership and Initiative: The response does not show any leadership skills or initiative.
8. Results and Impact: The student does not mention any

In [133]:
print(p3_q1_a5)

Student's Answer Score: 3/10

Feedback:
The student's response lacks detail and doesn't provide a good understanding of their role in the project, the specific challenges they faced, or how they handled them. It also doesn't show any demonstration of skills or experiences relevant to the Data Engineer - AI/ML role at S&P Global. The response also fails to mention any outcomes or achievements, making it difficult to assess their competency or performance. Furthermore, the student's answer indicates a lack of initiative and adaptability, which are important qualities for the role in question.

Improved Answer:

In a previous role as a data engineer, I was tasked with the challenge of developing an AI model for predicting customer churn for a telecom company. I was the lead data engineer responsible for the entire lifecycle of the project, from data collection and cleaning to implementing machine learning algorithms.

The project was challenging due to the large volume of data and the com

## Question 3 & Answer 7

In [135]:
# prompt 1/2/3, question 3, example answer 7

p1_q3_a7 = prompt_result.iloc[5,4]

p2_q3_a7 = prompt_result.iloc[8,4]

p3_q3_a7 = prompt_result.iloc[11,4]

In [137]:
print(p1_q3_a7)

Relevance to Question: 10/10
The answer directly addresses the question asked, providing insight into how the candidate stays motivated and manages their time effectively.

Technical Skills: 5/10
While the answer demonstrates a good understanding of time management strategies and their application, it does not provide any detail about the candidate's technical skills related to AI/ML or software development.

Problem-Solving Abilities: 8/10
The candidate's use of the Eisenhower Matrix and Pomodoro Technique does show an ability to solve problems related to time management and efficiency. However, it's unclear how they would approach more technical problems.

Communication Skills: 10/10
The answer is clear, concise, and well-structured. The candidate demonstrates good communication skills by effectively explaining their methods.

Cultural Fit: 7/10
The candidate's dedication to efficiency and their proactive approach to setting and achieving goals aligns with S&P Global's focus on provi

In [138]:
print(p2_q3_a7)

Student's answer score: 7/10

Feedback:
1. Specific Examples: The student provided a concrete example of using the Eisenhower Matrix and the Pomodoro Technique to manage time and stay motivated. This gives a clear picture of the methods they use. Excellent job on this aspect.
2. STAR Method: The student used this method to an extent, but could have been more specific about the situation and task that required the use of these techniques, and the actions taken to implement them.
3. Relevance to the Role: The student's answer is relevant for a Data Engineer role where time management and efficiency are key. However, they could have highlighted more about how these skills are especially relevant to data engineering work.
4. Problem-Solving Skills: The student did not explicitly mention problem-solving skills. These skills are crucial for a Data Engineer role, and should be added.
5. Teamwork and Collaboration: The student did not mention teamwork or collaboration, which are also important

In [139]:
print(p3_q3_a7)

Student's Answer Score: 8/10

Feedback:
The student's response is quite good as they demonstrate their ability to set goals, prioritize tasks, and manage time effectively. They also provide a quantifiable achievement, which is always beneficial in interviews. However, they could provide a more specific example to illustrate their approach in action, and could also tie their answer more directly to the role and company they are applying for. 

Improved Answer:
I draw motivation from setting challenging yet achievable goals. I use the Eisenhower Matrix to prioritize tasks according to their urgency and importance. This helps me focus on tasks that are vital for project success and delegate or schedule less critical tasks appropriately. I also use the Pomodoro Technique for focused work sessions, breaking tasks into 25-minute intervals with short breaks in between. This helps me maintain high productivity levels without burning out. 

For example, while working on a complex AI model at my

# Save data
answer_evaluation_prompt_result.csv

In [140]:
#prompt_result
prompt_result.to_csv("/content/drive/MyDrive/Capstone Project - Mock Interview App/answer_evaluation_prompt_result.csv", index=False)

In [141]:
prompt_result

,model,question_answer_combination,variable_combination,template,output
0,gpt_4,1,1,\nYou are an expert evaluator of interview res...,Relevance to Question: \nScore: 10/10 \nFeedba...
1,gpt_4,2,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe candidate's ...
2,gpt_4,3,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 9/10\nThe candidate dir...
3,gpt_4,5,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 7/10\nThe candidate doe...
4,gpt_4,12,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 8/10\nThe candidate doe...
5,gpt_4,23,1,\nYou are an expert evaluator of interview res...,Relevance to Question: 10/10\nThe answer direc...
6,gpt_4,1,1,"\nAs an experienced career coach, you help stu...",Score: 8/10\n\nFeedback: \n1. Specific Example...
7,gpt_4,5,1,"\nAs an experienced career coach, you help stu...",Student's Answer Score: 2/10\n\nFeedback:\n1. ...
8,gpt_4,23,1,"\nAs an experienced career coach, you help stu...",Student's answer score: 7/10\n\nFeedback:\n1. ...
9,gpt_4,1,1,"\nAs an experienced career coach, you help stu...",Student's Answer Score: 7/10\n\nFeedback with ...


# **Continue at 1_5_3_refined_answer_evaluation_prompt_behavioral_question.ipynb**